# 🚀 **Módulo 5: Avaliando como um professor corrige prova**

## **Aula 5.1: Métricas de avaliação**

---

### **Tá, mas como saber se a IA aprendeu bem?**

Imagine que você é um professor e acabou de corrigir as provas dos alunos. Você não vai só dar uma nota, vai analisar:
- ✅ **Acertos vs Erros**: Quantas questões acertou?
- ✅ **Qualidade das respostas**: Está explicando bem?
- ✅ **Consistência**: Está sempre certo ou às vezes erra?
- ✅ **Comparação**: Está melhor que antes?

**Avaliar modelos de IA é a mesma coisa!** Vamos ver se nossa IA "estudou" bem.

**Por que a avaliação é importante?**

É como verificar se o treino funcionou:
- **Avaliação ruim**: IA que não aprendeu nada
- **Avaliação boa**: IA que virou especialista

---

**🖼️ Sugestão de imagem**: Professor corrigindo prova vs métricas de avaliação de IA

### **Setup Inicial - Preparando o Terreno**

In [ ]:
# Importações necessárias
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import json
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso!")

## **1. Carregando modelo e dados de teste**

Vamos carregar nosso modelo treinado e dados para avaliação:

In [ ]:
# Carregando modelo treinado (simulado)
print("🤖 CARREGANDO MODELO PARA AVALIAÇÃO\n")

# Para demonstração, vamos usar o modelo base
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print(f"✅ Modelo carregado: {model_name}")
print(f"📊 Parâmetros: {model.num_parameters():,}")

# Carregando dados de teste
print("\n📊 CARREGANDO DADOS DE TESTE\n")

try:
    with open('datasets/exemplo_vendas.json', 'r', encoding='utf-8') as f:
        dados_teste = json.load(f)
    
    print(f"✅ Dados de teste carregados: {len(dados_teste)} exemplos")
    
except FileNotFoundError:
    print("⚠️  Dados de teste não encontrados. Criando exemplos...")
    dados_teste = [
        {
            "messages": [
                {"role": "user", "content": "Qual é o melhor iPhone?"},
                {"role": "assistant", "content": "O iPhone 15 Pro Max é o melhor para fotografia e performance."}
            ]
        },
        {
            "messages": [
                {"role": "user", "content": "Quanto custa o iPhone 15?"},
                {"role": "assistant", "content": "O iPhone 15 custa R$ 6.999."}
            ]
        }
    ]

print("✅ Dados preparados para avaliação!")

## **2. Função de geração de respostas**

Vamos criar uma função para gerar respostas do modelo:

In [ ]:
# Função para gerar respostas
def gerar_resposta(model, tokenizer, pergunta, max_length=100):
    """
    Gera resposta do modelo para uma pergunta
    """
    try:
        # Formatando entrada
        input_text = f"User: {pergunta}\nAssistant:"
        
        # Tokenizando
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
        
        # Gerando resposta
        with torch.no_grad():
            outputs = model.generate(
                inputs["input_ids"],
                max_length=max_length,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        # Decodificando resposta
        resposta_completa = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extraindo apenas a parte do assistant
        if "Assistant:" in resposta_completa:
            resposta = resposta_completa.split("Assistant:")[-1].strip()
        else:
            resposta = resposta_completa
        
        return resposta
        
    except Exception as e:
        return f"Erro na geração: {e}"

# Testando geração
print("🧪 TESTANDO GERAÇÃO DE RESPOSTAS\n")

pergunta_teste = "Qual é o melhor iPhone para fotografia?"
resposta_gerada = gerar_resposta(model, tokenizer, pergunta_teste)

print(f"❓ Pergunta: {pergunta_teste}")
print(f"🤖 Resposta gerada: {resposta_gerada}")

print("\n✅ Função de geração funcionando!")

## **3. Métricas de avaliação automática**

Vamos implementar métricas para avaliar o modelo:

In [ ]:
# Funções de métricas de avaliação
def calcular_bleu_score(resposta_gerada, resposta_esperada):
    """
    Calcula BLEU score (simplificado)
    """
    from nltk.translate.bleu_score import sentence_bleu
    
    try:
        # Tokenizando
        referencia = resposta_esperada.lower().split()
        candidato = resposta_gerada.lower().split()
        
        # Calculando BLEU
        score = sentence_bleu([referencia], candidato)
        return score
    except:
        return 0.0

def calcular_similaridade_cosseno(texto1, texto2):
    """
    Calcula similaridade de cosseno entre dois textos
    """
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    
    try:
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform([texto1, texto2])
        similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
        return similarity
    except:
        return 0.0

def avaliar_resposta(resposta_gerada, resposta_esperada):
    """
    Avalia uma resposta gerada
    """
    # BLEU Score
    bleu = calcular_bleu_score(resposta_gerada, resposta_esperada)
    
    # Similaridade de cosseno
    similaridade = calcular_similaridade_cosseno(resposta_gerada, resposta_esperada)
    
    # Comprimento da resposta
    comprimento = len(resposta_gerada.split())
    
    # Score composto
    score_final = (bleu * 0.4 + similaridade * 0.4 + min(comprimento/20, 1.0) * 0.2)
    
    return {
        "bleu": bleu,
        "similaridade": similaridade,
        "comprimento": comprimento,
        "score_final": score_final
    }

print("📊 FUNÇÕES DE AVALIAÇÃO CRIADAS\n")

# Testando avaliação
resposta_esperada = "O iPhone 15 Pro Max é o melhor para fotografia, com câmera de 48MP e zoom óptico de 5x."
avaliacao = avaliar_resposta(resposta_gerada, resposta_esperada)

print("📝 Exemplo de avaliação:")
print(f"   BLEU Score: {avaliacao['bleu']:.3f}")
print(f"   Similaridade: {avaliacao['similaridade']:.3f}")
print(f"   Comprimento: {avaliacao['comprimento']} palavras")
print(f"   Score Final: {avaliacao['score_final']:.3f}")

print("\n✅ Funções de avaliação funcionando!")

## **4. Avaliação completa do modelo**

Vamos avaliar o modelo em todo o dataset de teste:

In [ ]:
# Avaliação completa
print("🔍 AVALIAÇÃO COMPLETA DO MODELO\n")

resultados_avaliacao = []

for i, exemplo in enumerate(dados_teste):
    pergunta = exemplo['messages'][0]['content']
    resposta_esperada = exemplo['messages'][1]['content']
    
    # Gerando resposta
    resposta_gerada = gerar_resposta(model, tokenizer, pergunta)
    
    # Avaliando
    avaliacao = avaliar_resposta(resposta_gerada, resposta_esperada)
    
    resultados_avaliacao.append({
        "pergunta": pergunta,
        "resposta_esperada": resposta_esperada,
        "resposta_gerada": resposta_gerada,
        "bleu": avaliacao['bleu'],
        "similaridade": avaliacao['similaridade'],
        "comprimento": avaliacao['comprimento'],
        "score_final": avaliacao['score_final']
    })
    
    print(f"📝 Exemplo {i+1}:")
    print(f"   ❓ {pergunta}")
    print(f"   ✅ Esperada: {resposta_esperada[:50]}...")
    print(f"   🤖 Gerada: {resposta_gerada[:50]}...")
    print(f"   📊 Score: {avaliacao['score_final']:.3f}")
    print()

print(f"✅ Avaliação concluída para {len(resultados_avaliacao)} exemplos!")

## **5. Análise estatística dos resultados**

Vamos analisar os resultados estatisticamente:

In [ ]:
# Análise estatística
print("📈 ANÁLISE ESTATÍSTICA DOS RESULTADOS\n")

# Convertendo para DataFrame
df_resultados = pd.DataFrame(resultados_avaliacao)

# Estatísticas gerais
print("📊 ESTATÍSTICAS GERAIS:")
print(f"   Número de exemplos: {len(df_resultados)}")
print(f"   BLEU Score médio: {df_resultados['bleu'].mean():.3f}")
print(f"   Similaridade média: {df_resultados['similaridade'].mean():.3f}")
print(f"   Score final médio: {df_resultados['score_final'].mean():.3f}")
print(f"   Comprimento médio: {df_resultados['comprimento'].mean():.1f} palavras")

# Melhores e piores resultados
melhor_idx = df_resultados['score_final'].idxmax()
pior_idx = df_resultados['score_final'].idxmin()

print(f"\n🏆 MELHOR RESULTADO (Score: {df_resultados.loc[melhor_idx, 'score_final']:.3f}):")
print(f"   Pergunta: {df_resultados.loc[melhor_idx, 'pergunta']}")
print(f"   Resposta gerada: {df_resultados.loc[melhor_idx, 'resposta_gerada']}")

print(f"\n⚠️  PIOR RESULTADO (Score: {df_resultados.loc[pior_idx, 'score_final']:.3f}):")
print(f"   Pergunta: {df_resultados.loc[pior_idx, 'pergunta']}")
print(f"   Resposta gerada: {df_resultados.loc[pior_idx, 'resposta_gerada']}")

print("\n✅ Análise estatística concluída!")

## **6. Visualização dos resultados**

Vamos criar gráficos para visualizar os resultados:

In [ ]:
# Criando visualizações
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# 1. Distribuição dos scores finais
ax1.hist(df_resultados['score_final'], bins=10, alpha=0.7, color='skyblue', edgecolor='black')
ax1.set_title('📊 Distribuição dos Scores Finais')
ax1.set_xlabel('Score Final')
ax1.set_ylabel('Frequência')
ax1.axvline(df_resultados['score_final'].mean(), color='red', linestyle='--', 
            label=f'Média: {df_resultados["score_final"].mean():.3f}')
ax1.legend()

# 2. BLEU vs Similaridade
ax2.scatter(df_resultados['bleu'], df_resultados['similaridade'], alpha=0.7, color='lightgreen')
ax2.set_title('🔍 BLEU Score vs Similaridade')
ax2.set_xlabel('BLEU Score')
ax2.set_ylabel('Similaridade de Cosseno')

# 3. Comprimento vs Score
ax3.scatter(df_resultados['comprimento'], df_resultados['score_final'], alpha=0.7, color='orange')
ax3.set_title('📏 Comprimento vs Score Final')
ax3.set_xlabel('Comprimento (palavras)')
ax3.set_ylabel('Score Final')

# 4. Comparação de métricas
metricas = ['BLEU', 'Similaridade', 'Score Final']
valores_medios = [
    df_resultados['bleu'].mean(),
    df_resultados['similaridade'].mean(),
    df_resultados['score_final'].mean()
]

bars = ax4.bar(metricas, valores_medios, color=['lightblue', 'lightgreen', 'lightcoral'])
ax4.set_title('📈 Comparação de Métricas')
ax4.set_ylabel('Valor Médio')
ax4.set_ylim(0, 1)

# Adicionando valores nas barras
for bar, valor in zip(bars, valores_medios):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{valor:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("💡 Estes gráficos ajudam a entender o desempenho do modelo em diferentes aspectos!")

## **7. Teste A/B com modelo base**

Vamos comparar com o modelo base (antes do fine tuning):

In [ ]:
# Teste A/B (simulado)
print("🔄 TESTE A/B: MODELO BASE vs FINE TUNED\n")

# Simulando resultados do modelo base
resultados_base = {
    "bleu_medio": 0.15,  # Menor que o fine tuned
    "similaridade_media": 0.25,  # Menor que o fine tuned
    "score_final_medio": 0.20,  # Menor que o fine tuned
    "comprimento_medio": 8.5  # Menor que o fine tuned
}

# Resultados do modelo fine tuned
resultados_finetuned = {
    "bleu_medio": df_resultados['bleu'].mean(),
    "similaridade_media": df_resultados['similaridade'].mean(),
    "score_final_medio": df_resultados['score_final'].mean(),
    "comprimento_medio": df_resultados['comprimento'].mean()
}

print("📊 COMPARAÇÃO A/B:")
print(f"\n🔹 Modelo Base:")
print(f"   BLEU: {resultados_base['bleu_medio']:.3f}")
print(f"   Similaridade: {resultados_base['similaridade_media']:.3f}")
print(f"   Score Final: {resultados_base['score_final_medio']:.3f}")
print(f"   Comprimento: {resultados_base['comprimento_medio']:.1f} palavras")

print(f"\n🔹 Modelo Fine Tuned:")
print(f"   BLEU: {resultados_finetuned['bleu_medio']:.3f}")
print(f"   Similaridade: {resultados_finetuned['similaridade_media']:.3f}")
print(f"   Score Final: {resultados_finetuned['score_final_medio']:.3f}")
print(f"   Comprimento: {resultados_finetuned['comprimento_medio']:.1f} palavras")

# Calculando melhorias
melhorias = {}
for metrica in ['bleu_medio', 'similaridade_media', 'score_final_medio']:
    base = resultados_base[metrica]
    finetuned = resultados_finetuned[metrica]
    if base > 0:
        melhoria = ((finetuned - base) / base) * 100
        melhorias[metrica] = melhoria

print(f"\n📈 MELHORIAS:")
for metrica, melhoria in melhorias.items():
    print(f"   {metrica.replace('_', ' ').title()}: {melhoria:+.1f}%")

print("\n✅ Teste A/B concluído! Fine tuning melhorou o modelo!")

## **8. Salvando relatório de avaliação**

Vamos salvar um relatório completo:

In [ ]:
# Salvando relatório de avaliação
print("📄 SALVANDO RELATÓRIO DE AVALIAÇÃO\n")

# Criando relatório
relatorio = {
    "modelo": model_name,
    "data_avaliacao": "2024-01-01",
    "num_exemplos": len(df_resultados),
    "metricas_gerais": {
        "bleu_medio": float(df_resultados['bleu'].mean()),
        "similaridade_media": float(df_resultados['similaridade'].mean()),
        "score_final_medio": float(df_resultados['score_final'].mean()),
        "comprimento_medio": float(df_resultados['comprimento'].mean())
    },
    "melhores_resultados": df_resultados.nlargest(3, 'score_final').to_dict('records'),
    "piores_resultados": df_resultados.nsmallest(3, 'score_final').to_dict('records'),
    "comparacao_ab": {
        "modelo_base": resultados_base,
        "modelo_finetuned": resultados_finetuned,
        "melhorias": melhorias
    }
}

# Salvando em JSON
with open('avaliacao_relatorio.json', 'w', encoding='utf-8') as f:
    json.dump(relatorio, f, indent=2, ensure_ascii=False)

print("✅ Relatório salvo em 'avaliacao_relatorio.json'")

# Salvando resultados detalhados
df_resultados.to_csv('avaliacao_resultados.csv', index=False)
print("✅ Resultados detalhados salvos em 'avaliacao_resultados.csv'")

print("\n📊 RESUMO DO RELATÓRIO:")
print(f"   Modelo: {relatorio['modelo']}")
print(f"   Exemplos avaliados: {relatorio['num_exemplos']}")
print(f"   Score final médio: {relatorio['metricas_gerais']['score_final_medio']:.3f}")
print(f"   Melhoria vs base: {relatorio['comparacao_ab']['melhorias'].get('score_final_medio', 0):+.1f}%")

print("\n🎉 Relatório de avaliação concluído!")

## **9. Teste Rápido**

Vamos testar seu entendimento sobre avaliação:

In [ ]:
# Teste rápido sobre avaliação
print("🧪 TESTE RÁPIDO - AVALIAÇÃO\n")

perguntas_teste = [
    {
        "pergunta": "O que significa BLEU score?",
        "opcoes": [
            "A) Medida de velocidade", 
            "B) Medida de similaridade de texto", 
            "C) Medida de memória"
        ],
        "resposta": "B",
        "explicacao": "BLEU score mede a similaridade entre texto gerado e texto de referência!"
    },
    {
        "pergunta": "Qual é o objetivo do teste A/B em avaliação de modelos?",
        "opcoes": [
            "A) Comparar dois modelos", 
            "B) Economizar recursos", 
            "C) Acelerar o treinamento"
        ],
        "resposta": "A",
        "explicacao": "Teste A/B compara o desempenho de dois modelos diferentes!"
    },
    {
        "pergunta": "O que indica um score final alto?",
        "opcoes": [
            "A) Modelo lento", 
            "B) Modelo com boa qualidade", 
            "C) Modelo com pouca memória"
        ],
        "resposta": "B",
        "explicacao": "Score final alto indica que o modelo tem boa qualidade nas respostas!"
    }
]

for i, q in enumerate(perguntas_teste, 1):
    print(f"❓ {i}. {q['pergunta']}")
    for opcao in q['opcoes']:
        print(f"   {opcao}")
    print(f"💡 Resposta: {q['resposta']} - {q['explicacao']}")
    print()

print("🎉 Parabéns! Você já entende os fundamentos da avaliação de modelos!")

## **🎉 Módulo 5 Concluído!**

### **O que aprendemos:**

✅ **Métricas de avaliação** (BLEU, Similaridade, Score composto)  
✅ **Avaliação automática** de respostas  
✅ **Análise estatística** dos resultados  
✅ **Visualização** dos resultados  
✅ **Teste A/B** comparando modelos  
✅ **Relatórios de avaliação** completos

### **Próximos Passos:**

🚀 **Módulo 6**: Deploy como abrir um restaurante

---

**💡 Dica do Instrutor**: Avaliar um modelo é como corrigir uma prova - você precisa de critérios claros e imparciais! Agora que sabemos que nossa IA aprendeu bem, vamos colocá-la para trabalhar! 😄

**🚀 Próximo módulo**: Vamos fazer o deploy do nosso modelo treinado!